In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [5]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [6]:
# take one copy for model development data
data = model_data.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2833) (10802, 2833)


In [8]:
 features_list = [
'TIME_SINCE_FIRST(raw.created_timestamp WHERE hour_part2 = 1)#ever',
'NUM_UNIQUE(pg_success_raw_data.YEAR(created_time_ist))',
'days_since_first_premium_bank_trans',
'NUM_UNIQUE(pg_success_raw_data.bank_title)',
'MEAN(raw.amount WHERE mode_cleaned = 2)#ever',
'max_one_time_spent',
'AVG_TIME_BETWEEN(raw.created_timestamp WHERE mode_cleaned = 2)#ever',
'distinct_merchant',
'days_since_first_ecom_trans',
'days_on_pg',

'month_year','Weight_vintage',
'G15_B24']

In [9]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3165248/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3165248/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 11), (13738, 11), (20606,), (13738,))

In [12]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 11) (4122, 11) (13738, 11)


In [13]:
#Get vinatge weight values 
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [14]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [15]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [16]:
# pg
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
     'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 8,
    'alpha': 5.8,
    'gamma': 6.4,
    'scale_pos_weight': 1,
    'random_state': 24,
}
evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 80,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54934	eval-auc:0.59344
[10]	train-auc:0.55954	eval-auc:0.60351
[20]	train-auc:0.56384	eval-auc:0.60703
[30]	train-auc:0.56537	eval-auc:0.61411
[40]	train-auc:0.56816	eval-auc:0.61564
[50]	train-auc:0.56899	eval-auc:0.61745
[60]	train-auc:0.56974	eval-auc:0.61708
[70]	train-auc:0.56974	eval-auc:0.61708
[79]	train-auc:0.56974	eval-auc:0.61708


In [17]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.14, KS: 0.11
Eval AUC: 0.61, Gini: 0.21, KS: 0.18
Test AUC: 0.57, Gini: 0.13, KS: 0.11
OOT AUC: 0.55, Gini: 0.11, KS: 0.09


In [18]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
9          0.092937         0.088272        0.093515        0.084249    269   
8          0.089996         0.083925        0.084128        0.079739   2789   
7          0.082869         0.074213        0.079717        0.069925   1255   
6          0.080882         0.067347        0.069917        0.066076   1088   
5          0.084730         0.064171        0.066070        0.062764   1074   
4          0.073869         0.061352        0.062709        0.059822   1083   
3          0.067405         0.058486        0.059803        0.057385   1083   
2          0.043086         0.055310        0.057368        0.052814    998   
1          0.057610         0.051775        0.052731        0.050996   1163   

        label_good  label_bad  
Decile                         
9              244         25  
8             2538        251  
7             1151        104  
6             1000         88  
5              983         91  
4             1003         80  
3             1010         73  
2              955         43  
1             1096         67

In [16]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.086744         0.080114        0.080674        0.075154   4911   
6          0.064303         0.067936        0.073672        0.064521   1664   
5          0.054667         0.062488        0.064452        0.060938   1500   
4          0.064231         0.060326        0.060892        0.059321   1697   
3          0.053763         0.058673        0.059021        0.057437   1674   
2          0.065491         0.057123        0.057146        0.057103   1588   
1          0.046667         0.055455        0.056756        0.055285   3450   

        label_good  label_bad  
Decile                         
7             4485        426  
6             1557        107  
5             1418         82  
4             1588        109  
3             1584         90  
2             1484        104  
1             3289        161

In [17]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.078979         0.079606        0.080674        0.070848   4115   
6          0.068130         0.066591        0.070842        0.064160   1321   
5          0.059034         0.062231        0.062929        0.060938   1118   
4          0.055398         0.060325        0.060892        0.059321   1408   
3          0.052671         0.058675        0.059021        0.057437   1348   
2          0.044444         0.057123        0.057146        0.057103   1395   
1          0.049456         0.055443        0.056756        0.055285   3033   

        label_good  label_bad  
Decile                         
7             3790        325  
6             1231         90  
5             1052         66  
4             1330         78  
3             1277         71  
2             1333         62  
1             2883        150

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         5.0
4                                         days_on_pg         4.0
1                days_since_first_premium_bank_trans         1.0
2       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
3                                 max_one_time_spent         1.0


In [19]:
feature_importance_df

Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         5.0
4                                         days_on_pg         4.0
1                days_since_first_premium_bank_trans         1.0
2       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
3                                 max_one_time_spent         1.0

In [19]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_PG_Train_Pred_1524_Vin.csv',index=False)
Eval.to_csv('LeoPayu_PG_Eval_Pred_1524_Vin.csv',index=False)
Test.to_csv('LeoPayu_PG_Test_Pred_1524_Vin.csv',index=False)
Oot.to_csv('LeoPayu_PG_Oot_Pred_1524_Vin.csv',index=False)


In [20]:
#Save the model
xgb_model.save_model('LeoPayu_pg_15_in_24mob_Vin.json')
print('Model Saved')

Model Saved
